In [13]:
import requests
import pandas as pd
import re

prenset = "出席"
absent = "缺席"
leave = "請假"

In [2]:
url = "https://script.googleusercontent.com/macros/echo?user_content_key=CS1xGtawxafbsz_VXLceDZ8YhZJ3WLRGo3cTurMdyLFpRky5qzeXoqZBrYuJ0ElreSkNapo1i9TRXgBoQpou4xhHPHjfNUj4m5_BxDlH2jW0nuo2oDemN9CCS2h10ox_1xSncGQajx_ryfhECjZEnNGpsruz596piGbFJMaeWeZjFnhN0a9d4HwmOCyefi3ex5PPcpNsPlxGCUpGq7rwIA3gVkC6Pkbl8Md7CuyY-jH4-8QW02051A&lib=McNoStEX3P6w1EymfWUjOXqNKe58B53cM"
resp = requests.get(url)
resp_json = resp.json()

In [32]:
def getDf_from_google_sheet_json(json):
    df = pd.DataFrame(resp_json['data'][1:], columns=resp_json['data'][0])
    strings = df.columns
    pattern = r"\[(.*?)\]"
    result = []
    for string in strings:
        matches = re.findall(pattern, string)
        result.extend(matches)
    df.columns = list(df.columns[:2]) + result
    df['date'] = pd.to_datetime(df['日期']).dt.strftime('%Y-%m-%d')
    cols = df.columns.tolist()[2:]
    cols = [cols[-1]] + cols[:-1]
    output_df = df[cols].copy()
    return output_df

In [47]:
df = getDf_from_google_sheet_json(resp_json)
df

,date,Su,SzeKi,Caitlyn,Yau,Yannis,Yoyo,Heiting,Chuilam,Chautung
0,2023-06-03,出席,,出席,,,出席,出席,,出席
1,2023-05-27,出席,,出席,,,出席,出席,,出席
2,2023-06-10,出席,,出席,,,出席,出席,,出席
3,2023-06-17,出席,,出席,,,缺席,出席,,出席
4,2023-06-24,出席,,出席,,,出席,缺席,,出席
5,2023-07-02,出席,請假,請假,出席,缺席,出席,出席,出席,出席
6,2023-07-09,缺席,請假,出席,出席,出席,缺席,缺席,缺席,出席
7,2023-07-16,出席,出席,出席,出席,出席,出席,出席,出席,出席
8,2023-07-23,出席,出席,出席,出席,出席,出席,出席,請假,出席
9,2023-07-30,出席,出席,出席,出席,出席,出席,出席,出席,出席


In [37]:
def getDf_present_rate(df, prenset, absent, leave):
    result = pd.DataFrame()
    for column in df.columns[2:]:
        present_count = df[column].value_counts().get(prenset, 0)
        absent_count = df[column].value_counts().get(absent, 0)
        leave_count = df[column].value_counts().get(leave, 0)
        total_count = present_count + absent_count + leave_count
        present_rate = (present_count / total_count * 100) if total_count > 0 else 0
        result.loc[column, 'Present Count'] = present_count
        result.loc[column, 'Absent Count'] = absent_count
        result.loc[column, 'Leave Count'] = leave_count
        result.loc[column, 'Present Rate (%)'] = present_rate
    return result

In [38]:
result_df = getDf_present_rate(df, prenset, absent, leave)
result_df

,Present Count,Absent Count,Leave Count,Present Rate (%)
SzeKi,22.0,1.0,5.0,78.571429
Caitlyn,33.0,0.0,1.0,97.058824
Yau,21.0,5.0,3.0,72.413793
Yannis,25.0,4.0,0.0,86.206897
Yoyo,27.0,4.0,3.0,79.411765
Heiting,20.0,9.0,5.0,58.823529
Chuilam,16.0,9.0,4.0,55.172414
Chautung,31.0,1.0,2.0,91.176471
